# Temporal Difference Learning
## I. reinforcement learning的基本概念
1. online planning和offline planning \
① **offline planning** \
知道transition function和rewards function，可以直接求最优策略。MDP就是典型的offline planning。\
② **online planning** \
不知道transition function和rewards function，无法直接求最优策略。agent需要通过exploration来获取环境信息。

2. sample and episode \
① **sample**: 在online planning中，agent的一次行为反馈过程$(s, a, s^{'}, r)$称为一个sample。 \
② **episode**: 在online planning中，agent持续take action，并collect samples，直到达到terminal state的整个过程称为一个episode。通常要经历多个episods才能完成exploration。

3. model-based learning and model-free learning \
① <font color=blue>**model-based learning**</font>：先利用exploration得到的episodes数据，来构造model估计transition function和rewards function，然后就可以将问题转化为MDP问题求解。\
a. 估计transition function：$\hat T(s, a, s^{'})=\frac{\#(s, a, s^{'})}{\#(s, a)}$ \
b. 估计reward function：$\hat R(s, a, s^{'})= R(s, a, s^{'})$ \
② <font color=blue>**model-free learning**</font>：不估计transition function和rewards function，而是直接估计value或者Q-values of states.比如MC和TD。\
应用场景来看：model based model保留所有可能的$(s, a, s^{'})$table需要的存储成本很大，所以model-based learning多用在存储能力足够的场景下。
4. passive RL和active RL \
① 在passive RL中，会先给agent一个policy，然后agent用这个policy来经历episodes，以learn环境中的状态信息。TD就是passive RL方法。 \
② 在active RL中，没有固定的policy，agent会随时根据learn到的环境信息来更新policy。也就是exploration和exploiting同时进行。Q-learning就是active RL方法。

## II.basic TD：TD learning of state value
### II.1 思路：用MC抽样+RM迭代计算$V^{\pi}(s)$，<font color=red>做policy evaluation</font>
和MC exploring starts方法相似，也是在不知道Transition和Reward函数的条件下，将每一步中间action-reward作为一个样本，用抽样的方法来估$V^{\pi}(s)$，做policy evaluation。只是改用RM算法的迭代方式来估$V^{\pi}(s)$。

### II.2 算法
#### II.2.1 basic TD算法：做policy evaluation
iterate: \
$$
\begin{align} 
sample\_v(s_t)
& = r(s_t,a_t,s_{t+1}) + \gamma v^{\pi}_t(s_{t+1})\\
& = r_{t+1} + \gamma v^{\pi}_t(s_{t+1})\\
v^{\pi }_{t+1}(s_t) 
& \leftarrow v_t^{\pi}(s_t)-\alpha_t \left [v_t^{\pi}(s_t)-sample\_v(s_t)\right ]\\
& = v_t^{\pi}(s_t)-\alpha_t \left [v_t^{\pi}(s_t)-\left ( r_{t+1} + \gamma v^{\pi}_t(s_{t+1}) \right ) \right ]\\
v_{t+1}^{\pi}(s)& \leftarrow v^{\pi}_t(s), \forall s\ne s_t
\end{align}
$$
<font color=orange>符号说明：t时刻状态记为$s_t$。给定策略$\pi$，t时刻对状态s的state value的估计记为$v^{\pi}_t(s)$</font>

#### II.2.2 用RM推导basic TD的迭代形式
1. 原问题转化为RM问题：$g(w)=w-V^{\pi}(s)=w-E[R+\gamma V^{\pi}(s^{'})|s]=0$
2. t时刻的状态记为$s_t$，对应样本：$(s_t, a_t, s_{t+1}, r_{t+1})$
3. 构造迭代要素：
$$
\begin{align} 
\tilde g(w, \eta) 
& = w-sample\_v^{\pi}(s) \\
\eta & = \tilde g(w, \eta) - g(w) \\
& = [w-sample\_v^{\pi}(s)]-[w-V^{\pi}(s)] \\
& = V^{\pi}(s)-sample\_v^{\pi}(s)
\end{align}
$$

4. 得到迭代式
$$
\begin{align} 
w_{t+1} 
& = w_t - \alpha_t\tilde g(w_t, \eta) \\
& = w_t - \alpha_t(w_t-sample\_v^{\pi}(s_t)) \\
将w_t替换成v_t^{\pi}(s_t):\\
v^{\pi}_{t+1}(s_t) &= v^{\pi}_t(s_t) - \alpha_t\left [ v^{\pi}_t(s_t)-sample\_v^{\pi}(s_t) \right ]  \\
& = v^{\pi}_t(s_t) - \alpha_t\left [ v^{\pi}_t(s_t)- \left  (r_{t+1} + \gamma v^{\pi}_t(s_{t+1})\right ) \right ] 
\end{align}
$$

#### II.2.3 收敛性证明
[略，详见math of RL 7.1.3]

### II.3 性质
#### II.3.1 迭代式分析
$$
\begin{align} 
\overset{新估计}{\overbrace{v^{\pi}_{t+1}(s_t)}} 
& = \underset{旧估计}{\underbrace{v^{\pi}_t(s_t)}} - \alpha_t\left [ \underset{TD\ error}{\underbrace{v^{\pi}_t(s_t)-(\overset{TD\ target\ value}{\overbrace{r_{t+1} + \gamma v^{\pi}_t(s_{t+1})} } )}}  \right ] \\
& = (1-\alpha_t)\underset{旧估计}{\underbrace{v^{\pi}_t(s_t)}}+\alpha_t (\underset{TD\ target\ value}{\underbrace{r_{t+1} + \gamma v^{\pi}_t(s_{t+1})} } )
\end{align}
$$

1. 新估计是旧估计和TD target value的加权平均。所谓target value的意思就是新的样本会让估计值从旧估计结果向这个value方向偏移，因此叫target value。
2. TD error也称为temperal-difference。它反映了时刻t对state value的估计值与新的单个样本获得的估计值的差异，也体现了新的单个样本提供的信息量大小。
3. 这个迭代的前提是给定策略$\pi$，不能用在策略不断变化的场景。

#### II.3.2 与MC learning对比
XXX


## III. Sarsa：TD learning of action value
### III.1 Sarsa algorithm

#### III.1.1 思路
**用MC抽样+RM迭代估计$Q^{\pi}(s, a)$，<font color=red>做policy evaluation</font>。<font color=green>与basic TD的区别就是把对$V^{\pi}(s)$的估计改成直接估计$Q^{\pi}(s, a)$。</font>**

#### III.1.2 算法
1. 迭代推导
$$
\begin{align} 
sample\_q(s_t, a_t)
& = r(s_t,a_t,s_{t+1}) + \gamma q^{\pi}_t(s_{t+1}, a_{t+1})\\
& = r_{t+1} + \gamma q^{\pi}_t(s_{t+1}, a_{t+1})\\
v^{\pi }_{t+1}(s_t) 
& \leftarrow v_t^{\pi}(s_t)-\alpha_t \left [v_t^{\pi}(s_t)-sample\_v(s_t)\right ]\\
& = v_t^{\pi}(s_t)-\alpha_t \left [v_t^{\pi}(s_t)-\left ( r_{t+1} + \gamma q^{\pi}_t(s_{t+1}, a_{t+1}) \right ) \right ]\\
q_{t+1}^{\pi}(s)& \leftarrow q^{\pi}_t(s), \forall s\ne s_t
\end{align}
$$

2. 算法伪码
<img src="./pics/sarsa.png" alt="alt text" width="560"/>
注意本例中的迭代目标是找到从starting state到target state的最优路径，所以迭代条件的设置为有所不同。

### III.2 n-step Sarsa algorithm
#### III.2.1 思路：MC和Sarsa的折中

#### III.1.2 算法
1. 迭代推导
$$

$$

## IV. Q-learning: TD learning of optimal action value
### IV.1 Q-learning algorithm
#### IV.1.1 思路
1. 回顾Sarsa的算法思路：Sarsa在找最优策略的时候要分成两步，先做policy evaluation，再做policy improvement。原因是它的迭代式是在保持策略$\pi$不变的条件下估计action value$Q^{\pi}(s, a)$。得到action value的估计值之后再做policy improvement。
2. Q-learning的出发点：<font color=blue>**直接用RM算法的迭代方式找optimal action value。**</font>

#### IV.1.2 算法
1. 迭代推导
$$
\begin{align} 
sample\_q(s_t, a_t)
& = r(s_t,a_t,s_{t+1}) + \gamma \underset{a}{max}q^{\pi}_t(s_{t+1}, a)\\
& = r_{t+1} + \gamma \underset{a}{max}q^{\pi}_t(s_{t+1}, a)\\
q_{t+1}(s_t, a_t) 
& \leftarrow q_{t}(s_t, a_t)-\alpha_t \left [q_{t}(s_t, a_t)-sample\_v(s_t)\right ]\\
& = q_{t}(s_t, a_t)-\alpha_t \left [q_{t}(s_t, a_t)-\left ( r_{t+1} + \gamma \underset{a}{max}q^{\pi}_t(s_{t+1}, a)\right ) \right ]\\
q_{t+1}^{\pi}(s, a)& \leftarrow q^{\pi}_t(s, a), \forall (s, a)\ne (s_t, a_t)  \\
\end{align}
$$

2. **和Sarsa的取样方式差异**: sarsa在每次迭代的时候要用$(s_t, a_t, r_{t+1}, s_{t+1}, a_{t+1})$，但Q-learning只用$(s_t, a_t, r_{t+1}, s_{t+1})$

### IV.2 off-policy 和 on-policy learning
#### IV.2.1 概念定义
1. behavior policy和target policy \
(1) behavior policy：用来generate experience samples的策略。 \
(2) target policy：constantly update to converge to an optimal policy. 
2. on-policy learning和off-policy learning：\
如果一个算法中的behavior policy和target policy用的是相同的policy，该算法就是on-policy learning。反之，就是off-policy learning。
#### IV.2.2 off-policy的优点
用别的策略生成的样本来learn最优策略会带来一些好处。比如：
1. 这个被学习的策略可以是人类的操作。
2. <font color=red>hebavior policy可以故意设为有很强的exploration能力的policy，这会提升模型learn最优策略的能力。</font>因为，估计action value要对所有state-action pair估计$Q(s,a)$。这意味着每个pair都需要大量sample。如果target policy本身没有很强的exploration的能力，就很难做到。比如Sarsa用$\epsilon -greedy$来保证一定的exploration ability，但$\epsilon$的值通常设置得很小，exploration的效果有限。但是如果能用一个有很强的exploration能力的策略专门来generate episodes，然后用off-policy learning来学习最优策略，那么learning效率会有大幅提升。

#### IV.2.3 典型的off-policy 和 on-policy learning算法
1. on-policy：Sarsa，因为每次做了policy improvement之后就用同一个policy来抽样做该policy的evaluation。
2. off-policy：Q-learning，因为

#### IV.2.2 Q-learning的两种形式
1. 一般的Q-learning是on-policy learning
<img src="./pics/qlearningon.png" alt="alt text" width="560"/>

2. Q-learning也可以改造成off-policy learning的形式
<img src="./pics/qlearningoff.png" alt="alt text" width="560"/>

### III.4 Approximate Q-learning

#### MC
1. 求解目标：$E[R_{t+1}+\gamma R_{t+2}+\gamma^2R_{t+3}+...]$
2. 迭代样本：$r_{t+1}+\gamma r_{t+2}+\gamma^2r_{t+3}+...$
#### Sarsa
1. 求解目标：$E[R_{t+1}+\gamma Q(s_{t+1}, a_{t+1})|s_t, a_t]$
2. 迭代样本：$r_{t+1}+\gamma q_t(s_{t+1}, a_{t+1})$
#### n-step Sarsa
1. 求解目标：$E[R_{t+1}+\gamma R_{t+2}+...+\gamma^n Q(s_{t+n}, a_{t+n})|s_t, a_t]$
2. 迭代样本：$r_{t+1}+\gamma r_{t+2}+...+\gamma^n q_t(s_{t+n}, a_{t+n})$
#### Q-learning
1. 求解目标：$E[R_{t+1}+\gamma max_a Q(s_{t+1}, a)|s_t, a_t]$
2. 迭代样本：$r_{t+1}+\gamma max_aq_t(s_{t+1}, a)$